In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
# Configuration
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = 'PlantVillage'  

In [5]:
def create_model(num_classes):
    """Create a CNN model with transfer learning"""
    base_model = keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model
    base_model.trainable = False
    
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

In [7]:
def prepare_data(data_dir):
    """Prepare training and validation data"""
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2,
        fill_mode='nearest'
    )
    
    train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )
    
    val_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, val_generator


In [9]:
def plot_history(history):
    """Plot training history"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(history.history['accuracy'], label='Train Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Val Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)
    
    ax2.plot(history.history['loss'], label='Train Loss')
    ax2.plot(history.history['val_loss'], label='Val Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    print("Training history plot saved as 'training_history.png'")


In [11]:
def train_model():
    """Main training function"""
    print("Preparing data...")
    train_gen, val_gen = prepare_data(DATA_DIR)
    
    num_classes = len(train_gen.class_indices)
    print(f"Number of classes: {num_classes}")
    print(f"Classes: {list(train_gen.class_indices.keys())}")
    
    print("\nBuilding model...")
    model = create_model(num_classes)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("\nModel Summary:")
    model.summary()
    
    # Callbacks
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7
        ),
        keras.callbacks.ModelCheckpoint(
            'crop_disease_model_best.h5',
            monitor='val_accuracy',
            save_best_only=True
        )
    ]
    
    print("\nTraining model...")
    history = model.fit(
        train_gen,
        epochs=EPOCHS,
        validation_data=val_gen,
        callbacks=callbacks
    )
    
    # Save final model
    model.save('crop_disease_model.h5')
    print("\nModel saved as 'crop_disease_model.h5'")
    
    # Save class labels
    import json
    class_labels = {v: k for k, v in train_gen.class_indices.items()}
    with open('class_labels.json', 'w') as f:
        json.dump(class_labels, f)
    print("Class labels saved as 'class_labels.json'")
    
    # Plot training history
    plot_history(history)
    
    return model, history


In [ ]:
if __name__ == "__main__":
    # Check if data directory exists
    if not os.path.exists(DATA_DIR):
        print(f"Error: Data directory '{DATA_DIR}' not found!")
        print("\nPlease download the PlantVillage dataset from Kaggle:")
        print("https://www.kaggle.com/datasets/emmarex/plantdisease")
        print("Extract it and update the DATA_DIR variable in this script.")
    else:
        model, history = train_model()
        print("\nTraining complete!")
        print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

Preparing data...
Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
Number of classes: 15
Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']

Building model...

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 15)                  │           3,855 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,589,775 (9.88 MB)

 Trainable params: 331,791 (1.27 MB)

...

517/517 ━━━━━━━━━━━━━━━━━━━━ 848s 2s/step - accuracy: 0.8349 - loss: 0.4847 - val_accuracy: 0.8855 - val_loss: 0.3469 - learning_rate: 0.0010
Epoch 7/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 964s 2s/step - accuracy: 0.8430 - loss: 0.4639 - val_accuracy: 0.8848 - val_loss: 0.3318 - learning_rate: 0.0010
Epoch 8/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8463 - loss: 0.4475

517/517 ━━━━━━━━━━━━━━━━━━━━ 1134s 2s/step - accuracy: 0.8420 - loss: 0.4638 - val_accuracy: 0.8967 - val_loss: 0.3121 - learning_rate: 0.0010
Epoch 9/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 1237s 2s/step - accuracy: 0.8483 - loss: 0.4452 - val_accuracy: 0.8901 - val_loss: 0.3165 - learning_rate: 0.0010
Epoch 10/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 889s 2s/step - accuracy: 0.8501 - loss: 0.4312 - val_accuracy: 0.8908 - val_loss: 0.3193 - learning_rate: 0.0010
Epoch 11/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 841s 2s/step - accuracy: 0.8526 - loss: 0.4302 - val_accuracy: 0.8906 - val_loss: 0.3283 - learning_rate: 0.0010
Epoch 12/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8624 - loss: 0.3997

517/517 ━━━━━━━━━━━━━━━━━━━━ 880s 2s/step - accuracy: 0.8622 - loss: 0.3972 - val_accuracy: 0.9003 - val_loss: 0.2907 - learning_rate: 5.0000e-04
Epoch 13/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8732 - loss: 0.3837

517/517 ━━━━━━━━━━━━━━━━━━━━ 875s 2s/step - accuracy: 0.8725 - loss: 0.3780 - val_accuracy: 0.9115 - val_loss: 0.2737 - learning_rate: 5.0000e-04
Epoch 14/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8726 - loss: 0.3875

517/517 ━━━━━━━━━━━━━━━━━━━━ 879s 2s/step - accuracy: 0.8709 - loss: 0.3810 - val_accuracy: 0.9151 - val_loss: 0.2542 - learning_rate: 5.0000e-04
Epoch 15/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 878s 2s/step - accuracy: 0.8781 - loss: 0.3634 - val_accuracy: 0.9073 - val_loss: 0.2724 - learning_rate: 5.0000e-04
Epoch 16/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 909s 2s/step - accuracy: 0.8724 - loss: 0.3721 - val_accuracy: 0.9078 - val_loss: 0.2705 - learning_rate: 5.0000e-04
Epoch 17/20
517/517 ━━━━━━━━━━━━━━━━━━━━ 848s 2s/step - accuracy: 0.8743 - loss: 0.3654 - val_accuracy: 0.9064 - val_loss: 0.2636 - learning_rate: 5.0000e-04
Epoch 18/20
266/517 ━━━━━━━━━━━━━━━━━━━━ 7:01 2s/step - accuracy: 0.8832 - loss: 0.3385